# 0. 전체 Notebook 코드 (VSCode/Jupyter에서 바로 쓰기 좋은 형태)

In [14]:
# %% [markdown]
# # pepbind_pipeline 단계별 디버그 노트북
# - 기존 run 결과 폴더(pipeline/RUN_ID)를 기준으로
# - ColabFold → Prodigy → Vina → PLIP → (최종 스코어) 를
#   단계별로 따로 실행/재실행하기 위한 노트북 템플릿입니다.
# %%
from pathlib import Path
import importlib

import pepbind_pipeline as pb  # pepbind_pipeline.py 가 이 노트북과 같은 프로젝트 안에 있다고 가정

# pepbind_pipeline 수정 후에는 이 셀을 다시 실행해서 최신 코드로 reload
pb = importlib.reload(pb)

# 디버그하고 싶은 run ID (폴더 이름)만 바꿔서 사용
RUN_ID = "PDP_20251122_221946"  # ← 여기만 바꿔주면 됨

[INFO] PyTorch device: cuda


In [15]:
# %%
# 기본 경로 탐색 유틸 (pipeline/RUN_ID 를 현재 작업 디렉토리 주변에서 자동 탐색)

def find_run_dir(run_id: str) -> Path:
    cwd = Path.cwd().resolve()

    candidate_dirs = [
        cwd / "pipeline" / run_id,
        cwd.parent / "pipeline" / run_id,
        cwd.parent.parent / "pipeline" / run_id,
        cwd / run_id,  # 혹시 바로 아래에 RUN_ID 폴더가 있을 수도 있으니
    ]

    checked = []
    print("===== RUN_DIR 후보 경로 체크 =====")
    for p in candidate_dirs:
        p = p.resolve()
        if p in checked:
            continue
        checked.append(p)
        print(f"{p} : {'OK' if p.is_dir() else 'NOT FOUND'}")
        if p.is_dir():
            return p

    raise FileNotFoundError(
        f"RUN_ID={run_id} 에 해당하는 폴더를 찾지 못했습니다.\n"
        f"위에 출력된 경로를 보고, 실제 위치를 확인한 뒤 아래에서 RUN_DIR 를 수동으로 지정해 주세요."
    )

try:
    RUN_DIR = find_run_dir(RUN_ID)
except FileNotFoundError as e:
    print("\n자동 탐색 실패. 필요하면 아래 주석을 풀고 수동 지정하세요.")
    # 예시:
    # RUN_DIR = Path("/home/aisys/work/pipeline") / RUN_ID
    raise

print("\n===== 사용 RUN_DIR =====")
print(RUN_DIR)

===== RUN_DIR 후보 경로 체크 =====
/home/aisys/work/pipeline/pipeline/PDP_20251122_221946 : NOT FOUND
/home/aisys/work/pipeline/PDP_20251122_221946 : OK

===== 사용 RUN_DIR =====
/home/aisys/work/pipeline/PDP_20251122_221946


In [16]:
# %%
# 서브 디렉토리 경로 설정

FASTA_DIR      = RUN_DIR / "fasta"
PDB_DIR        = RUN_DIR / "pdb"
COLABFOLD_DIR  = PDB_DIR / "colabfold_output"
RESULTS_DIR    = RUN_DIR / "results"
TEMP_DIR       = RUN_DIR / "temp"

for path in [RUN_DIR, FASTA_DIR, PDB_DIR, COLABFOLD_DIR, RESULTS_DIR, TEMP_DIR]:
    print(f"{path} : {'OK' if path.exists() else 'MISSING'}")

/home/aisys/work/pipeline/PDP_20251122_221946 : OK
/home/aisys/work/pipeline/PDP_20251122_221946/fasta : OK
/home/aisys/work/pipeline/PDP_20251122_221946/pdb : OK
/home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output : OK
/home/aisys/work/pipeline/PDP_20251122_221946/results : OK
/home/aisys/work/pipeline/PDP_20251122_221946/temp : OK


In [17]:
# %%
# 기본 상태 확인용 헬퍼 (중간 중간 상태 보고 싶을 때 호출)

def show_basic_status():
    print("===== 기본 경로 상태 =====")
    print("RUN_DIR      :", RUN_DIR, "OK" if RUN_DIR.exists() else "MISSING")
    print("FASTA_DIR    :", FASTA_DIR, "OK" if FASTA_DIR.exists() else "MISSING")
    print("PDB_DIR      :", PDB_DIR, "OK" if PDB_DIR.exists() else "MISSING")
    print("COLABFOLD_DIR:", COLABFOLD_DIR, "OK" if COLABFOLD_DIR.exists() else "MISSING")
    print("RESULTS_DIR  :", RESULTS_DIR, "OK" if RESULTS_DIR.exists() else "MISSING")
    print("TEMP_DIR     :", TEMP_DIR, "OK" if TEMP_DIR.exists() else "MISSING")

    if COLABFOLD_DIR.exists():
        pdb_list = list(COLABFOLD_DIR.glob("*.pdb"))
        print("  └─ ColabFold PDB 개수:", len(pdb_list))
    else:
        print("  └─ ColabFold PDB 없음")

show_basic_status()

===== 기본 경로 상태 =====
RUN_DIR      : /home/aisys/work/pipeline/PDP_20251122_221946 OK
FASTA_DIR    : /home/aisys/work/pipeline/PDP_20251122_221946/fasta OK
PDB_DIR      : /home/aisys/work/pipeline/PDP_20251122_221946/pdb OK
COLABFOLD_DIR: /home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output OK
RESULTS_DIR  : /home/aisys/work/pipeline/PDP_20251122_221946/results OK
TEMP_DIR     : /home/aisys/work/pipeline/PDP_20251122_221946/temp OK
  └─ ColabFold PDB 개수: 10


# 1. pepbind_pipeline 안에 어떤 함수들이 있는지 빠르게 훑어보기

In [18]:
# %% [markdown]
# ## 1. pepbind_pipeline 안에 어떤 함수들이 있는지 빠르게 훑어보기
# - 이름에 colab / fasta / peptide / prodigy / vina / plip / score 등이 들어간 함수들을 찾아서
#   실제 함수명을 확인합니다.
# - 만약 내가 아래 셀에서 호출한 함수 이름과 조금이라도 다르면
#   이 셀 결과를 보고 함수 이름만 맞게 수정해주면 됩니다.

# %%
keywords = ["colab", "fasta", "peptide", "prodigy", "vina", "plip", "score", "rank"]
for key in keywords:
    names = [n for n in dir(pb) if key.lower() in n.lower()]
    if not names:
        continue
    print(f"\n[{key}] 관련 함수/변수")
    for n in names:
        attr = getattr(pb, n)
        kind = "function" if callable(attr) else type(attr).__name__
        print(f" - {n} ({kind})")


[colab] 관련 함수/변수
 - COLABFOLD_CMD (str)
 - RUN_COLABFOLD (bool)
 - prepare_colabfold_batch_csv (function)
 - run_colabfold_batch_with_progress (function)

[fasta] 관련 함수/변수
 - write_peptide_fasta (function)
 - write_target_fasta (function)

[peptide] 관련 함수/변수
 - NUM_PEPTIDES (int)
 - PEPTIDE_LENGTH (int)
 - generate_peptides_with_mlm (function)
 - write_peptide_fasta (function)

[prodigy] 관련 함수/변수
 - PRODIGY_SCRIPT (str)
 - PRODIGY_SUMMARY_COLS (list)
 - RUN_PRODIGY (bool)
 - load_prodigy_scores (function)
 - parse_prodigy_dg_from_stdout (function)
 - run_prodigy_on_rank1 (function)

[vina] 관련 함수/변수
 - RUN_VINA (bool)
 - VINA_CMD (str)
 - VINA_SUMMARY_COLS (list)
 - load_vina_scores (function)
 - parse_vina_score_from_stdout (function)
 - run_vina_on_rank1 (function)

[plip] 관련 함수/변수
 - PLIP_CMD (str)
 - PLIP_SUMMARY_COLS (list)
 - RUN_PLIP (bool)
 - load_plip_scores (function)
 - run_plip_on_rank1 (function)

[score] 관련 함수/변수
 - load_iptm_scores (function)
 - load_plip_scores (functio

# 2. (선택) FASTA/펩타이드 생성 단계만 따로 돌리고 싶을 때

In [ ]:
# %% [markdown]
# ## 2. (선택) FASTA/펩타이드 생성 단계만 따로 돌리고 싶을 때
# - 실제로는 pepbind_pipeline.py 안에 있는 함수 이름/인자에 맞게 수정해야 합니다.
# - ColabFold 이전 단계에서 에러가 날 때 이 셀을 이용해 해당 부분만 테스트할 수 있습니다.
#
# 예시(함수 이름은 사용자 코드에 맞게 바꿔야 함):
#
# pb.generate_peptides_and_fastas(
#     run_dir=RUN_DIR,
#     target_fasta_path=FASTA_DIR / "target_protein.fasta",
#     peptides_fasta_path=FASTA_DIR / "peptides.fasta",
#     ...
# )

# %%
print("이 셀은 템플릿입니다. pepbind_pipeline.py 안의 FASTA/펩타이드 생성 함수를 여기에서 직접 호출해서 사용하세요.")

# 3. (선택) ColabFold 단계만 따로 실행

In [ ]:
# %% [markdown]
# ## 3. (선택) ColabFold 단계만 따로 실행
# - 마찬가지로, pepbind_pipeline.py 에서 ColabFold 호출하는 함수를 그대로 가져다 쓰면 됩니다.
# - 이미 ColabFold 결과가 잘 나와 있다면 이 셀은 건너뛰어도 됩니다.

# %%
print("이 셀도 템플릿입니다. ColabFold 실행 함수가 있다면 여기에서만 따로 호출해서 테스트할 수 있습니다.")
# 예시 (실제 함수명/인자는 pepbind_pipeline.py 에 맞게 수정 필요):
# pb.run_colabfold_for_run(
#     fasta_dir=FASTA_DIR,
#     out_dir=COLABFOLD_DIR,
#     model_type="alphafold2_multimer_v3",
#     num_models=5,
#     ...
# )

# 4. ColabFold 결과에서 rank_001 PDB만 모으기

In [19]:
# %% [markdown]
# ## 4. ColabFold 결과에서 rank_001 PDB만 모으기
# - 기존 pepbind_pipeline.py 에 collect_rank1_pdbs 같은 함수가 있다면 그대로 사용
# - 없다면 노트북 안에서 간단히 glob 패턴으로 구현

# %%
from typing import List

if hasattr(pb, "collect_rank1_pdbs"):
    collect_rank1_pdbs = pb.collect_rank1_pdbs  # 기존 함수 재사용
    print("pepbind_pipeline.collect_rank1_pdbs 함수를 사용합니다.")
else:
    def collect_rank1_pdbs(colabfold_dir: Path) -> List[Path]:
        pattern = "complex_*rank_001*.pdb"
        pdbs = sorted(colabfold_dir.glob(pattern))
        print(f"pattern='{pattern}' 로 찾은 rank_001 PDB 개수: {len(pdbs)}")
        return pdbs
    print("collect_rank1_pdbs 함수가 없어, 노트북 내의 간단한 glob 버전을 사용합니다.")

if not COLABFOLD_DIR.exists():
    raise RuntimeError("COLABFOLD_DIR 가 존재하지 않습니다. ColabFold 결과가 있는 run ID 인지 확인하세요.")

rank1_pdbs = collect_rank1_pdbs(COLABFOLD_DIR)
print(f"\nrank_001 PDB 총 개수: {len(rank1_pdbs)}")
for p in rank1_pdbs[:5]:
    print(" -", p.name)

collect_rank1_pdbs 함수가 없어, 노트북 내의 간단한 glob 버전을 사용합니다.
pattern='complex_*rank_001*.pdb' 로 찾은 rank_001 PDB 개수: 10

rank_001 PDB 총 개수: 10
 - complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
 - complex_4_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb


# 5. Vina 단계만 따로 실행

In [20]:
# %% [markdown]
# ## 6. Vina 단계만 따로 실행
# - 방금 수정한 run_vina_on_rank1(rank1_pdbs, vina_dir: Path)를 그대로 사용
# - 이미 rank_001 PDB 리스트를 확보했으므로, 여기서는 ColabFold를 다시 돌릴 필요가 없음
# - Vina 관련 코드만 수정하고 싶을 때 이 셀만 계속 돌리면 됨

# %%
VINA_DIR = RESULTS_DIR / "vina"
VINA_DIR.mkdir(parents=True, exist_ok=True)
print("VINA_DIR:", VINA_DIR)

# pepbind_pipeline.py 에 들어 있는 run_vina_on_rank1 함수를 그대로 사용
pb.run_vina_on_rank1(rank1_pdbs, VINA_DIR)

vina_summary = VINA_DIR / "vina_summary.xlsx"
if vina_summary.exists():
    print("✅ vina_summary.xlsx 생성 완료:", vina_summary)
else:
    print("⚠ vina_summary.xlsx 를 찾지 못했습니다. 함수 내부에서 파일 이름을 확인해 보세요.")

VINA_DIR: /home/aisys/work/pipeline/PDP_20251122_221946/results/vina

STEP 4: AutoDock Vina 도킹

[INFO] Vina 준비: complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
[INFO] complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb 체인 구성: {'A': 222, 'B': 4}
[INFO] 자동 할당 체인: receptor=A, ligand=B
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdb -xr -opdbqt -O /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_receptor_A.pdbqt
[RUN] /home/aisys/miniconda3/envs/pepbind/bin/obabel -ipdb /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_m

In [21]:
p = Path("/home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output/complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb")
print(p)
counts = pb.get_chain_residue_counts(p)
print("체인별 residue 개수:")
for cid, n in counts.items():
    print(f"  chain {cid!r}: {n} residues")

# 전체 rank_001 PDB에 대해서도 보고 싶으면:
# for pdb in COLABFOLD_DIR.glob("complex_*rank_001*.pdb"):
#     counts = get_chain_residue_counts(pdb)
#     print(pdb.name, "→", counts)

/home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output/complex_6_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb
체인별 residue 개수:
  chain 'A': 222 residues


# 6. PLIP 단계만 따로 실행

In [22]:
# %% [markdown]
# ## 7. PLIP 단계만 따로 실행
# - run_plip_on_rank1(rank1_pdbs, plip_dir) 형태를 가정
# - 함수 이름/인자는 pepbind_pipeline.py 에 맞게 필요시 수정

# %%
PLIP_DIR = RESULTS_DIR / "plip"
PLIP_DIR.mkdir(parents=True, exist_ok=True)
print("PLIP_DIR:", PLIP_DIR)

if hasattr(pb, "run_plip_on_rank1"):
    pb.run_plip_on_rank1(rank1_pdbs, PLIP_DIR)
else:
    print("pepbind_pipeline.run_plip_on_rank1 함수를 찾지 못했습니다.")
    print("→ pepbind_pipeline.py 안 PLIP 실행 함수 이름을 확인해서 여기서 직접 호출해 주세요.")

plip_summary = PLIP_DIR / "plip_summary.xlsx"
if plip_summary.exists():
    print("✅ plip_summary.xlsx 생성 완료:", plip_summary)
else:
    print("⚠ plip_summary.xlsx 를 찾지 못했습니다. 함수 내부에서 파일 이름을 확인해 보세요.")

PLIP_DIR: /home/aisys/work/pipeline/PDP_20251122_221946/results/plip

STEP 5: PLIP 상호작용 분석
[RUN] plip -f /home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb -o /home/aisys/work/pipeline/PDP_20251122_221946/results/plip/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000 -x -t
✔️ PLIP 완료: complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb → /home/aisys/work/pipeline/PDP_20251122_221946/results/plip/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000
[RUN] plip -f /home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output/complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb -o /home/aisys/work/pipeline/PDP_20251122_221946/results/plip/complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000 -x -t
✔️ PLIP 완료: complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb → /home/aisys/work/pipeline

# 7. Prodigy 단계만 따로 실행

In [23]:
# %% [markdown]
# ## 5. Prodigy 단계만 따로 실행
# - run_prodigy_on_rank1(rank1_pdbs, prodigy_dir) 형태의 함수를 가정
# - pepbind_pipeline.py 에서 함수 이름이 다르면 이 셀에서만 이름 맞게 바꿔주면 됨
# - 이미 Prodigy 결과가 있는 경우에도, 함수 수정 후 이 셀만 다시 돌려서 재계산 가능

# %%
PRODIGY_DIR = RESULTS_DIR / "prodigy"
PRODIGY_DIR.mkdir(parents=True, exist_ok=True)
print("PRODIGY_DIR:", PRODIGY_DIR)

if hasattr(pb, "run_prodigy_on_rank1"):
    pb.run_prodigy_on_rank1(rank1_pdbs, PRODIGY_DIR)
else:
    print("pepbind_pipeline.run_prodigy_on_rank1 함수를 찾지 못했습니다.")
    print("→ pepbind_pipeline.py 안에서 Prodigy 실행하는 함수 이름을 확인해서, 이 셀에서 직접 호출해 주세요.")

prodigy_summary = PRODIGY_DIR / "prodigy_summary.xlsx"
if prodigy_summary.exists():
    print("✅ prodigy_summary.xlsx 생성 완료:", prodigy_summary)
else:
    print("⚠ prodigy_summary.xlsx 를 찾지 못했습니다. 함수 내부에서 파일 이름을 확인해 보세요.")

PRODIGY_DIR: /home/aisys/work/pipeline/PDP_20251122_221946/results/prodigy

STEP 6: PRODIGY 결합 친화도 평가
[RUN] prodigy /home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output/complex_0_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb --selection A B
[RUN] prodigy /home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output/complex_1_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb --selection A B
[RUN] prodigy /home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output/complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb --selection A B
[WARN] PRODIGY 접촉 없음: complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000. (로그: complex_2_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000_prodigy.txt)
[RUN] prodigy /home/aisys/work/pipeline/PDP_20251122_221946/pdb/colabfold_output/complex_3_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb --selection A B
[WARN] PRODIGY 접촉 없음: complex_3_unrelax

# 8. (선택) 최종 스코어/랭킹 통합 단계

In [24]:
# %% [markdown]
# ## 8. (선택) 최종 스코어/랭킹 통합 단계
# - Prodigy/Vina/PLIP/ColabFold 점수를 종합해서 최종 점수를 계산하는 함수가
#   pepbind_pipeline.py 안에 있다면 그 함수를 그대로 호출하면 됩니다.
# - 함수 이름을 모르겠다면, 1번 셀(함수 리스트 출력)을 참고해서 수정하세요.

# %%
def load_peptides_from_fasta(fasta_path: Path):
    """peptides.fasta 에서 펩타이드 서열 리스트만 뽑아오기 (간단 버전)"""
    peptides = []
    seq = ""
    with open(fasta_path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if seq:
                    peptides.append(seq)
                    seq = ""
            else:
                seq += line
        if seq:
            peptides.append(seq)
    return peptides

# pepbind_pipeline.py 의 init_workspace() 에서 쓰는 키 구조에 맞게 폴더 dict 구성
FOLDERS = {
    "root": RUN_DIR,
    "fasta": FASTA_DIR,
    "pdb": PDB_DIR,
    "colabfold_out": COLABFOLD_DIR,
    "results": RESULTS_DIR,
    "vina": RESULTS_DIR / "vina",
    "plip": RESULTS_DIR / "plip",
    "prodigy": RESULTS_DIR / "prodigy",
    "temp": TEMP_DIR,
}

PEPTIDES = load_peptides_from_fasta(FASTA_DIR / "peptides.fasta")

final_xlsx = pb.build_and_save_final_table(FOLDERS, PEPTIDES, rank1_pdbs)
print("최종 결과 엑셀:", final_xlsx)

print("\nRESULTS_DIR 안의 엑셀 파일 확인:")
for xlsx_path in sorted(RESULTS_DIR.glob("*.xlsx")):
    print(" -", xlsx_path.name)

[INFO] Vina 요약 엑셀에서 점수 로드: /home/aisys/work/pipeline/PDP_20251122_221946/results/vina/vina_summary.xlsx
[INFO] Vina 점수를 읽어온 구조 수: 10
[INFO] PRODIGY 요약 엑셀에서 점수 로드: /home/aisys/work/pipeline/PDP_20251122_221946/results/prodigy/prodigy_summary.xlsx
[INFO] PRODIGY ΔG를 요약 파일에서 불러옴: 10개 구조
[INFO] ipTM 값을 읽어온 구조 수: 10 / 10
[INFO] PLIP 파싱 디버그 로그: /home/aisys/work/pipeline/PDP_20251122_221946/results/plip/plip_parse_debug.txt
[INFO] PLIP 요약 엑셀 저장: /home/aisys/work/pipeline/PDP_20251122_221946/results/plip/plip_summary.xlsx
[INFO] PLIP 상호작용을 읽어온 구조 수: 10
✅ 최종 결과 엑셀 저장: /home/aisys/work/pipeline/PDP_20251122_221946/results/final_peptide_ranking_A_20251123_171614.xlsx
최종 결과 엑셀: /home/aisys/work/pipeline/PDP_20251122_221946/results/final_peptide_ranking_A_20251123_171614.xlsx

RESULTS_DIR 안의 엑셀 파일 확인:
 - final_peptide_ranking_A_20251123_125218.xlsx
 - final_peptide_ranking_A_20251123_171614.xlsx
